# Setup

## Imports

In [ ]:
rm(list = ls())
library(tidyverse)
library(data.table)
library(broom)
library(dtplyr)
library(lme4)
library(lmerTest)
library(ggbeeswarm)
library(cowplot)

# load("questionnaires.RData")
# save.image("questionnaires.RData")
source("https://raw.githubusercontent.com/hauselin/Rcode/master/cleanQuestionnaire.R")

## Load Data

In [ ]:
dataFile <- "Data/sm_questionnaires.csv"
qualtrics <- tbl_dt(fread(dataFile, header = T)[-1])

names(qualtrics)[duplicated(names(qualtrics))] # check duplicated column names
qualtrics[, n_distinct(ResponseID)]

# remove incomplete subjects
qualtrics$debrief = as.numeric(qualtrics$debrief)
qualtrics<-subset(qualtrics, (!is.na(qualtrics$debrief)))

qualtrics[, n_distinct(ResponseID)]

# create subject ID
qualtrics$subjectID = 1:qualtrics[, n_distinct(ResponseID)]

In [ ]:
wellBeing <- select(qualtrics, subjectID, starts_with("wb")) %>% 
    tbl_df() %>% 
    mutate_all(as.numeric) %>% 
    tbl_dt()

addictionProne <- select(qualtrics, subjectID, starts_with("addict")) %>% 
    tbl_df() %>% 
    mutate_all(as.numeric) %>% 
    tbl_dt()

scales <- list()
scalesWideRaw <- list()

# test for people choosing only one score
test_range = socialQs
which(apply(test_range[,-1], 1, range)[2,] - apply(test_range[,-1], 1, range)[1,] ==0)

# remove 56, 141, 209 (done at the end)